In [1]:
! pip install mlflow boto3 awscli

In [2]:
import mlflow
mlflow.set_tracking_uri("http://98.84.31.179:5000/")

In [16]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - BOW Trigram max_feature")
# mlflow.delete_experiment("Exp 3 - TfIdf Trigram max_features")

2024/12/15 18:16:13 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - BOW Trigram max_feature' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://dsmp-mlflow-bucket/572963331633213831', creation_time=1734266793727, experiment_id='572963331633213831', last_update_time=1734266793727, lifecycle_stage='active', name='Exp 3 - BOW Trigram max_feature', tags={}>

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [19]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [20]:
# Step 1: Function to run the experiment
def run_experiment_bow_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"BOW_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with BOW Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "BOW")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: BOW Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_BOW_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_bow_max_features(max_features)

2024/12/15 18:17:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_1000 at: http://98.84.31.179:5000/#/experiments/572963331633213831/runs/cea985e43b8f45b4b2f039472ebaaa30
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/572963331633213831


2024/12/15 18:18:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_2000 at: http://98.84.31.179:5000/#/experiments/572963331633213831/runs/2eed7d120c394f8ca89bcbb19fd111db
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/572963331633213831


2024/12/15 18:19:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run BOW_Trigrams_max_features_8000 at: http://98.84.31.179:5000/#/experiments/572963331633213831/runs/1aca2f37357c4d9497cb3809a888a5ba
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/572963331633213831


2024/12/15 18:26:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_9000 at: http://98.84.31.179:5000/#/experiments/572963331633213831/runs/2c511d708a664f0faa7d45d3313e6038
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/572963331633213831


2024/12/15 18:27:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_10000 at: http://98.84.31.179:5000/#/experiments/572963331633213831/runs/865f94a872e74ae8b50fd0fdfeae4053
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/572963331633213831
